In [1]:
import numpy as np
import pandas as pd
import mysql.connector


In [2]:
#save key from keys.txt
with open('keys.txt') as f:
    lines = f.readlines()
    key = lines[0].strip()


In [3]:
db = mysql.connector.connect(
    host="localhost",
    user="ramsus",
    passwd=key,
    database="pagos"
)


In [4]:
mycursor = db.cursor()

In [5]:
def print_sql(query):
    mycursor.execute(query)
    result = mycursor.fetchall()
    for x in result:
        print(x)

In [6]:
print_sql("DESCRIBE pagos")

('id', b'int', 'NO', 'PRI', None, 'auto_increment')
('paciente', b'varchar(35)', 'NO', '', None, '')
('valor', b'int', 'NO', '', None, '')
('actividad', b'varchar(35)', 'NO', '', None, '')
('fecha', b'timestamp', 'NO', '', None, '')
('nequi', b'tinyint(1)', 'NO', '', b'0', '')
('datafono', b'tinyint(1)', 'NO', '', b'0', '')
('transferencia', b'tinyint(1)', 'NO', '', b'0', '')
('efectivo', b'tinyint(1)', 'NO', '', b'0', '')
('notas', b'varchar(100)', 'YES', '', None, '')


In [7]:
# read csv and normaliza rows names
excel = pd.read_csv('pagos.csv', encoding='latin-1', sep=',')
excel.columns = excel.columns.str.replace(' ', '')
del excel['%']
excel.dropna(inplace=True)
# convert Si in True and No in False
excel['Nequi'] = excel['Nequi'].map({'Si': 1, 'No': 0})
excel['Datafono'] = excel['Datafono'].map({'Si': 1, 'No': 0})
excel['Transferencia'] = excel['Transferencia'].map({'Si': 1, 'No': 0})
excel['Efectivo'] = excel['Efectivo'].map({'Si': 1, 'No': 0})
# convert value to number and remove the $ symbol and .
excel['Valor'] = excel['Valor'].str.replace('$', '')
excel['Valor'] = excel['Valor'].str.replace(',', '')
excel['Valor'] = excel['Valor'].str.replace(' ', '')
excel['Valor'] = excel['Valor'].astype(int)
# convert date in format day/month/year to datetime
excel['Fecha'] = pd.to_datetime(excel['Fecha'], format='%d/%m/%Y')

excel.reset_index(drop=True, inplace=True)

excel

,Fecha,Paciente,Valor,Actividad,Nequi,Datafono,Transferencia,Efectivo
0,2023-06-09,Manfry Ariza,60000,Valoracion,0,1,0,0
1,2023-06-09,Nazly Avila,185000,Montaje/retiro,1,0,0,0
2,2023-06-09,Paula Hernandez,25000,Radiografia,0,0,0,1
3,2023-06-09,Kevin Santiago Soto,105000,Control/Reparaciones,0,0,0,1
4,2023-06-09,Laura Cupa,60000,Control,0,1,0,0
...,...,...,...,...,...,...,...,...
134,2023-07-01,Carmen Mojica,50000,Control,0,0,0,1
135,2023-07-01,Carmen Mojica,10000,Control,0,0,1,0
136,2023-07-01,Lizeth Vargas,230000,"Papada, Encimas",0,0,0,1
137,2023-07-01,Maria Paula Correa,100000,Placa Activada,0,1,0,0


In [9]:
# check changes
print_sql("SELECT * FROM pagos WHERE id > 138")

(139, ' Doctora Diana ', -595000, 'pago porcentaje', datetime.datetime(2023, 7, 1, 0, 0), 0, 0, 0, 1, None)
(158, ' Katherine Veloza ', 300000, 'bichectomia', datetime.datetime(2023, 6, 10, 0, 0), 0, 0, 0, 1, None)
(159, ' Katherine Veloza ', 370000, 'lipopapada', datetime.datetime(2023, 6, 10, 0, 0), 0, 0, 0, 1, None)
(160, ' Aura Yamile ', 300000, 'bichectomia', datetime.datetime(2023, 6, 10, 0, 0), 0, 1, 0, 0, None)
(161, ' Aura Yamile ', 480000, 'lipopapada', datetime.datetime(2023, 6, 10, 0, 0), 0, 1, 0, 0, None)
(162, ' Yolima Montero ', 300000, 'bichectomia', datetime.datetime(2023, 6, 14, 0, 0), 0, 1, 0, 0, None)
(163, ' Yolima Montero ', 550000, 'lipopapada', datetime.datetime(2023, 6, 14, 0, 0), 0, 1, 0, 0, None)


In [10]:
# close connection
db.close()